In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import torch.nn.functional as F
os.chdir("../")
from tqdm import tqdm
from src.utils.get_datasets import getDataTF, getDataHF
from src.data.processing_data import Format
from src.models.pipelines import Pipeline
from src.models.encoders import TransformersEncoder

2023-03-12 14:18:43.532170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 14:18:44.239478: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-12 14:18:44.239544: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-12 14:18:44.239551: W 

In [2]:
_dataset = "mrda"

In [3]:
df = pd.read_csv(f"./inputs_data/data_{_dataset}_train.csv", encoding="utf-8",  sep="|")
df.head()

,Utterance,Dialogue_ID,Label
0,i guess .,Bed002,0
1,okay we're on .,Bed002,0
2,so just make sure that th- - your wireless mik...,Bed002,0
3,check one .,Bed002,0
4,check one .,Bed002,0


In [4]:
# frequency of each dialog acts 
# df.value_counts("Label")

In [5]:
# number of distinct label i the dataset
len(df["Label"].unique())

5

In [6]:
# number of distinct dialogue i the dataset
len(df["Dialogue_ID"].unique())

56

In [7]:
# average number of utterance per dialogue
int(df[["Dialogue_ID", "Label"]].groupby("Dialogue_ID").count().Label.mean())

1498

In [8]:
dimDialogAct, Contexts, Labels = Format(_dataset, 50, "spread").get_contexts_labels()

2023-03-12 14:18:47.810609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 14:18:48.534663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13581 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5


In [9]:
Labels[0].shape

TensorShape([56, 50, 5])

In [10]:
len(Contexts[0][0])

50

In [11]:
# TransformersEncoder('xlnet-base-cased', "spread", 50).batch_embedding(Contexts[0])

## Testing Pipeline working

In [14]:
df_report = Pipeline(
    "swda",
    "spread",
    50,
    'bert-base-uncased',
    "GRU").summary_exec()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 998/998 [07:30<00:00,  2.21it/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cl

Epoch 1/500
63/63 [==============================] - 2s 16ms/step - loss: 0.0999 - val_loss: 0.0849
Epoch 2/500
63/63 [==============================] - 1s 11ms/step - loss: 0.0785 - val_loss: 0.0777
Epoch 3/500
63/63 [==============================] - 1s 12ms/step - loss: 0.0749 - val_loss: 0.0731
Epoch 4/500
63/63 [==============================] - 1s 12ms/step - loss: 0.0698 - val_loss: 0.0708
Epoch 5/500
63/63 [==============================] - 1s 11ms/step - loss: 0.0699 - val_loss: 0.0705
Epoch 6/500
63/63 [==============================] - 1s 11ms/step - loss: 0.0630 - val_loss: 0.0620
Epoch 7/500
63/63 [==============================] - 1s 12ms/step - loss: 0.0592 - val_loss: 0.0601
Epoch 8/500
63/63 [==============================] - 1s 11ms/step - loss: 0.0577 - val_loss: 0.0630
Epoch 9/500
63/63 [==============================] - 1s 11ms/step - loss: 0.0591 - val_loss: 0.0583
Epoch 10/500
63/63 [==============================] - 1s 11ms/step - loss: 0.0559 - val_loss: 0.0580

In [15]:
df_report

,dataset_name,encoder_model,decoder_model,performance
0,swda,bert-base-uncased,GRU,0.694545
